#### Notes

_Once the model is trained, it can be used automatically classify patches that were previously extracted (E)._

"Extracted patches were then passed to the trained classifier as input. The output for each patch was a corresponding vector representing the probability distribution of class categories to which the center-most pixel of the patch likely belonged. For each patch, the extracted location, the presumed class label, and the difference between the two highest probability distributions (i.e., top-1 and top-2 choices) were recorded.

The difference between the top two probabilities was considered the relative confidence level of the classifier when making the prediction. If the difference was small, the classifier was less confident about its top-1 choice (i.e., the presumed class label). By setting a confidence threshold value, sparse labels that the classifier was less certain about could be ignored. However, determining the ideal threshold involved trying different values and comparing the classification scores of the sparse labels predicted for the test images against the labels in the corresponding pixel indices of the manually created ground-truth dense labels (i.e., test set). As is discussed in the results section, the final threshold value that was chosen was a trade-off between the total number of labels that were accepted and their classifications scores.

With regards to efficiency, the CNN patch-based image classifier was able to assign roughly 200 sparse labels to an image per second, as opposed to the one annotation every 6 s that it cost users who used CPCe manually (Beijbom et al., 2015)."

![alt text](../Figures/getting_dense_labels.png)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import get_cmap
cm = get_cmap('jet')

import imgaug.augmenters as iaa
from keras.models import load_model

In [ ]:
class_categories = {'Branching' : 0, 
                  'Fish' : 1, 
                  'Massive' : 2,
                  'Not Massive' : 3,
                  'Substrate' : 4,
                  'Target' : 5,
                  'Water' : 6}

In [ ]:
image_file = "..\\Figures\\example_coral_patch.png"
image_name = os.path.basename(image_file).split(".")[0]

image = plt.imread(image_file)   

In [ ]:
patch_df = pd.read_csv(image_name + "_Sparse_Points.csv")

patches = []

for i, r in patch_df.iterrows():
    
    patch = plt.imread(r['file'])
    patch = iaa.Resize(224).augment_image(image=patch) * (1./255.0)
    patches.append(patch)

patches = np.array(patches)

In [ ]:
# Load up the best trained model
model = load_model("Best_Model_and_Weights.h5")

In [ ]:
# Making predictions on the patches that were extracted
predictions = model.predict(patches)

# Getting the predicted labels and also the confidence score between the top two likely classes
predicted_labels = [list(class_categories)[np.argmax(prediction, axis = 0)] for prediction in predictions]
confidence = [sorted(prediction)[-1] - sorted(prediction)[-2] for prediction in predictions]

# Adding the to the dataframe
patch_df['Label'] = predicted_labels
patch_df['Confidence'] = confidence

# Saving again
patch_df.to_csv(image_name + "_Sparse_Points.csv")

In [ ]:
# Just viewing the image with the sparse labels super imposed
additional_labels = patch_df[patch_df['Confidence'] >= .70]

X = additional_labels['X'].values
Y = additional_labels['Y'].values
C = [cm[_]/255.0 for _ in additional_labels['Labels'].values]

plt.figure(figsize = (10, 10))
plt.imshow(image)
plt.scatter(X, Y, c = C)
plt.xticks([])
plt.yticks([])